In [3]:
from helper_functions import *
import nengo
import nengo_dl

# Specifying architecture in Tensorflow

In [4]:
inp = tf.keras.Input(shape=(2))
dense1 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(inp)
dense2 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(dense1)
output = layers.Dense(1, use_bias = True)(dense2)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

# Loading the weights from the .onnx file to the TF model

In [6]:
onnx_model_path = '../benchmarks/controller_single_pendulum.onnx'
onnx_model = onnx.load(onnx_model_path)
params_array = []

for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    params_array.append(tensor_array)
    
weights_list=[]
bias_list=[]
i=0
for tensor_array in params_array:
    if(i%2==0):
        weights_list.append(tensor_array)
    i = i+1
i=0
for tensor_array in params_array:
    if(i%2==1):
        bias_list.append(tensor_array)
    i = i+1
    
weights_list.reverse()
bias_list.reverse()

for i in range(1, len(weights_list) + 1):
    transposed_weights = weights_list[i-1]
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

# Extracting weights from the .onnx file

In [11]:
weights, biases = extract_model_params_tf(nn_ctlr)

In [12]:
layer_no = 3
neuron_no = 1
time_steps = 5
input_bounds = [[1.0,1.2],[0.0,0.2]]

In [14]:
converter = nengo_dl.Converter(nn_ctlr)

scale_firing_rates=1000
activation=nengo.SpikingRectifiedLinear() # Specifying activation function to replace ReLU
# activation=nengo.RectifiedLinear() 
synapse=None

nengo_converter = nengo_dl.Converter(
        nn_ctlr,
        swap_activations={tf.nn.relu: activation},
        scale_firing_rates=scale_firing_rates,
        synapse=synapse,
    )
nengo_input = nengo_converter.inputs[inp]
nengo_output = nengo_converter.outputs[output]

## Evaluate a specific input with the SNN

In [15]:
net_inp = [3,5]
# Uncomment the cell below to evaluate the SNN with random inputs
# net_inp = [random.uniform(input_bounds[0][0], input_bounds[0][1]), random.uniform(input_bounds[1][0], input_bounds[1][1]), random.uniform(input_bounds[2][0], input_bounds[2][1]), random.uniform(input_bounds[3][0], input_bounds[3][1]), random.uniform(input_bounds[4][0], input_bounds[4][1])]
net_inp = [round(x, 2) for x in net_inp]
input_arr = np.array([net_inp])

snn_input = np.array([[net_inp]*time_steps])
with nengo_dl.Simulator(nengo_converter.net, progress_bar=False, seed = 0) as nengo_sim_toy:
    data = nengo_sim_toy.predict({nengo_input: snn_input})
out = data[nengo_output]

c:\users\kiit\nengo-dl\nengo_dl\simulator.py:456: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(


In [16]:
out

array([[[-3.4311547],
        [-4.895354 ],
        [-2.8145046],
        [-4.780682 ],
        [-3.526785 ]]], dtype=float32)

# Function to simulate the SNN with Gurobi

In [17]:
def simulate_with_gurobi(w,b,layer, no,time_steps, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    # Declarations
    for time in range(1,time_steps+1):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_{time} = model.addVar(lb=-GRB.INFINITY, name='A0_{num}_{time}')")
            equations.append(f"model.addConstr(A0_{num}_{time}=={input_bounds[0][time-1][num-1]})")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"X{layer}_{no}_{time} = model.addVar(name='X{layer}_{no}_{time}')")
        declare.append(f"P{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")
        declare.append(f"q{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{layer}_{no}_{time}')")
        declare.append(f"A{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{layer}_{no}_{time}')")
    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")

                equation = f'S{i}_{j}_{time} == ('
                for k in range(len(w[i-1][0])):
                    if(k!=0):
                        equation += f' + '
                    equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")

    for time in range(1,time_steps+1):
        equations.append(f"model.addConstr(P{layer}_{no}_{time} == P{layer}_{no}_{time-1} + S{layer}_{no}_{time})")

        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")

    equations.append(f'model.setObjective(0, gp.GRB.MAXIMIZE)')

    return equations, declare

# Function to set parameters and solve constraints with Gurobi

In [18]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings_SP_sim_random.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
    model=gp.Model("Encodings")
    model.Params.MIPFocus = focus
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1

    try:
        f = open(file_path,"r")
        try:
            for l in f:
                exec(l)
        finally:
            f.close()
    except IOError:
        pass
    model.optimize()
    return model

In [19]:
converter = nengo_dl.Converter(nn_ctlr)

scale_firing_rates=1000
activation=nengo.SpikingRectifiedLinear() # Specifying activation function to replace ReLU
# activation=nengo.RectifiedLinear() 
synapse=None

nengo_converter = nengo_dl.Converter(
        nn_ctlr,
        swap_activations={tf.nn.relu: activation},
        scale_firing_rates=scale_firing_rates,
        synapse=synapse,
    )
nengo_input = nengo_converter.inputs[inp]
nengo_output = nengo_converter.outputs[output]

# Code to evaluate and store random samples with the SNN

In [21]:
from openpyxl import Workbook
input_bounds = [[1.0,1.2],[0.0,0.2]]

layer_no = 3
neuron_no = 1
time_steps = 20

error_gurobi = 0
error_nengo = 0
error_gurobi_sq = 0
error_nengo_sq = 0
max_error_gurobi = 0
max_error_nengo = 0
samples = 500

# print("Time Steps: ",time_steps)
excel_file_path = f'SP_random_sims.xlsx'

workbook = Workbook()
sheet = workbook.active
sheet.cell(row=1, column=1, value='Sl. No.')
sheet.cell(row=1, column=2, value='Input')
sheet.cell(row=1, column=3, value='ANN Prediction')

row = 1
col = 4

for t in range (1, time_steps+1):
    
    sheet.cell(row=1, column=col, value=f'Nengo Pred_{t}_TS')
    sheet.cell(row=1, column=col+1, value=f'Gurobi Pred_{t}_TS')
    sheet.cell(row=1, column=col+2, value=f'Error_{t}_TS (ANN- Nengo SNN)')
    sheet.cell(row=1, column=col+3, value=f'Error_{t}_TS (ANN- Gurobi SNN)')
    sheet.cell(row=1, column=col+4, value=f'Error_sq_{t}_TS Nengo')
    sheet.cell(row=1, column=col+5, value=f'Error_sq_{t}_TS Gurobi')
    col +=7

for i in range(1,samples+1):
    
    net_inp = [random.uniform(input_bounds[0][0], input_bounds[0][1]), random.uniform(input_bounds[1][0], input_bounds[1][1])]
    net_inp = [round(x, 2) for x in net_inp]
    input_arr = np.array([net_inp])
    
    # Prediction from ANN controller using Keras
    ann_prediction = nn_ctlr.predict(input_arr)
    ann_prediction = [np.round(x, 2) for x in ann_prediction]
    sheet.cell(row=i+2, column=1, value=i)
    rounded_list = [round(element, 3) for element in net_inp]
    sheet.cell(row=i+2, column=2, value=str(rounded_list))
    sheet.cell(row=i+2, column=3, value=round(float(ann_prediction[0][0]),3))
    
    # Prediction from SNN simulated with Gurobi
    snn_input = np.array([[net_inp]*time_steps])
    eqn, dec = simulate_with_gurobi(weights, biases, layer_no, neuron_no, time_steps, snn_input)
    m = summon_gurobi(dec, eqn,0, 1)
    
    col = 5
    for j in range(1,time_steps+1):
        prefix = str(f'P{layer_no}_{neuron_no}_{j}')
        variables_with_prefix = [var for var in m.getVars() if var.varName.startswith(prefix)]
        snn_prediction = variables_with_prefix[0].X / j    
        sheet.cell(row=i+2, column=col, value=round(snn_prediction,6))
        
        error = ann_prediction[0][0] - snn_prediction
        sheet.cell(row=i+2, column=col+2, value=error)
        sheet.cell(row=i+2, column=col+4, value=error**2)
        col+=7
    m.dispose()
    
    
    if(i%50==0):
        print(i, "samples done!")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20
50 samples done!
100 samples done!
150 samples done!
200 samples done!


AttributeError: Unable to retrieve attribute 'X'

In [22]:
workbook.save(excel_file_path)
workbook.close()
print('Done')

Done
